In [9]:
"""
    Instruções:
        [v]- Download dos dados municipais do estado de Minas Gerais;
        [v]- Transformação dos dados de desmatamento;
        [v] Processamento dos dados:
            -reprojeção para EPSG:31983;
            -cálculo de área;
        [v] Geração dos arquivos de saída no formato GeoJSON;
"""

'\n    Instruções:\n        [v]- Download dos dados municipais do estado de Minas Gerais;\n        [v]- Transformação dos dados de desmatamento;\n        [v] Processamento dos dados:\n            -reprojeção para EPSG:31983;\n            -cálculo de área;\n        [v] Geração dos arquivos de saída no formato GeoJSON;\n'

In [1]:
import geopandas as gpd
import pandas as pd
import requests
from bs4 import BeautifulSoup

def obter_municipios_geojson(url, output_file):
    municipios = gpd.read_file(url)
    municipios = municipios.to_crs(epsg=31983)
    municipios["area_km2"] = municipios['geometry'].area / 10**6
    municipios.to_file(output_file, driver="GeoJSON")

def obter_dados_wikipedia(url):
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        tabela = soup.find('table')
        dados = pd.read_html(str(tabela), flavor='html5lib')[0]
        return dados
    else:
        print(f"Falha na requisição. Status code: {response.status_code}")
        return None

def combinar_dados(dados_geojson, dados1, dados2, output_file):
    dados1.columns = ['Posição', 'Município', 'Produto Interno Bruto Nominal (2020)', 'PIB Per Capita (2020)']
    dados2.columns = ['Posição', 'Município', 'População']

    dados_combinados = pd.merge(dados_geojson, dados1, left_on='name', right_on='Município', how='left')
    dados_combinados = pd.merge(dados_combinados, dados2, on='Município', how='left')

    dados_combinados.to_csv(output_file, index=False)
    print("Dados combinados e salvos com sucesso!")

url_geojson_municipios = "https://raw.githubusercontent.com/tbrugz/geodata-br/master/geojson/geojs-31-mun.json"
output_geojson_municipios = "dados/municipios-mg.geojson"
url_pib = "https://pt.wikipedia.org/wiki/Lista_de_munic%C3%ADpios_de_Minas_Gerais_por_PIB"
url_populacao = "https://pt.wikipedia.org/wiki/Lista_de_munic%C3%ADpios_de_Minas_Gerais_por_popula%C3%A7%C3%A3o"
output_csv_combinado = "dados/populacao_pib_municipios_mg.csv"

obter_municipios_geojson(url_geojson_municipios, output_geojson_municipios)
dados_pib = obter_dados_wikipedia(url_pib)
dados_populacao = obter_dados_wikipedia(url_populacao)

municipios_geojson = gpd.read_file(output_geojson_municipios)

if dados_pib is not None and dados_populacao is not None:
    combinar_dados(municipios_geojson, dados_pib, dados_populacao, output_csv_combinado)


Dados combinados e salvos com sucesso!


In [2]:
import pandas as pd
import seaborn as sns

def load_and_clean_dataset(file_path):
    dataset = pd.read_csv(file_path, sep=",")
    dataset.columns = ["municipio_id", "nome_municipio", "descricao", "area_km2", "geometria", "pos_x", "municipio", "pib_nominal", "pib_capita", "pos_y", "populacao"]

    colunas_interesse = ["geometria", "pos_x", "municipio", "pib_nominal", "pib_capita", "pos_y", "populacao"]
    dataset_sem_nulos = dataset.dropna(subset=colunas_interesse)

    colunas_remover = ["municipio", "descricao", "pos_x", "pos_y"]
    novo_dataset = dataset_sem_nulos.drop(columns=colunas_remover)
    return novo_dataset

def verifica_dados_repetidos(dados, coluna):
    agrupado = dados.groupby([coluna]).size()
    return agrupado

def ordenar_e_resetar_indice(dados):
    if "id" in dados.columns:
        dados = dados.sort_values(by='id') 
        # dados.reset_index(drop=True, inplace=True)
        return dados
    else:
        # dados.reset_index(drop=True, inplace=True)
        return dados


def main(db_pop, db_pop_ord):
    dataset_sem_nulos = load_and_clean_dataset(db_pop)

    dataset_ordenado = ordenar_e_resetar_indice(dataset_sem_nulos)

    dataset_ordenado.to_csv(db_pop_ord, index=False)

    print(f"Dados de focos de desmatamento salvos em {db_pop_ord}")

if __name__ == "__main__":
    output_pop = "dados/populacao_pib_municipios_mg.csv"
    output_pop_ord = "dados/populacao_pib_municipios_mg_ordenado.csv"
    
    main(output_pop, output_pop_ord)


Dados de focos de desmatamento salvos em dados/populacao_pib_municipios_mg_ordenado.csv


In [3]:
import geopandas as gpd
import pandas as pd
from pyproj import CRS, Transformer

def carregar_focos_desmatamento(file_path):
    return gpd.read_file(file_path)

def concatenar_e_transformar_geometria(gdf_list, crs_target):
    gdf_combined = gpd.GeoDataFrame(pd.concat(gdf_list, ignore_index=True), geometry="geometry")
    gdf_combined.crs = gdf_list[0].crs
    gdf_combined = gdf_combined.to_crs(crs_target)
    return gdf_combined

def salvar_geodataframe_como_geojson(gdf, output_path):
    gdf.to_file(output_path, driver="GeoJSON")

def main(file_aug, file_set, output_geojson):
    gdf_aug = carregar_focos_desmatamento(file_aug)
    gdf_set = carregar_focos_desmatamento(file_set)
    gdf_combined = concatenar_e_transformar_geometria([gdf_aug, gdf_set], crs_target="EPSG:31983")
    
    salvar_geodataframe_como_geojson(gdf_combined, output_geojson)

    print(f"Dados de focos de desmatamento salvos em {output_geojson}")

if __name__ == "__main__":
    file_aug = "dados/desmatamento_ago22.gpkg"
    file_set = "dados/desmatamento_set22.gpkg"
    output_geojson = "dados/focos-desmatamento-mg.geojson"
    main(file_aug, file_set, output_geojson)


Dados de focos de desmatamento salvos em dados/focos-desmatamento-mg.geojson
